In [1]:
import numpy as np

In [2]:
model_name = 'bge-base-en-v1.5-new'

In [3]:
Xids = np.load(f'./vectors/{model_name}/Xids.npy')
Xmasks = np.load(f'./vectors/{model_name}/Xmasks.npy')
labels = np.load(f'./vectors/{model_name}/labels.npy')

In [4]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

/Users/USER/Desktop/Side_project/transformer-classifier/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmasks, labels))

2024-02-11 22:26:26.754928: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-02-11 22:26:26.754946: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-02-11 22:26:26.754950: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-02-11 22:26:26.754982: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-11 22:26:26.754997: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
for i in dataset.take(1):
    print(i)

(<tf.Tensor: shape=(50,), dtype=float64, numpy=
array([  101.,  1037.,  2186.,  1997.,  9686., 17695., 18673., 14313.,
        1996., 15262.,  3351.,  2008.,  2054.,  2003.,  2204.,  2005.,
        1996., 13020.,  2003.,  2036.,  2204.,  2005.,  1996., 25957.,
        4063.,  1010.,  2070.,  1997.,  2029.,  5681.,  2572., 25581.,
        2021.,  3904.,  1997.,  2029.,  8310.,  2000.,  2172.,  1997.,
        1037.,  2466.,  1012.,   102.,     0.,     0.,     0.,     0.,
           0.,     0.])>, <tf.Tensor: shape=(50,), dtype=float64, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.])>, <tf.Tensor: shape=(3,), dtype=float64, numpy=array([1., 0., 0.])>)


In [7]:
def map_func(input_ids, masks, lables):
    return {'input_ids': input_ids, 'attention_mask': masks}, lables

In [8]:
dataset = dataset.map(map_func).shuffle(100000)

In [9]:
DS_LEN = len(list(dataset))

In [10]:
split = .8
train = dataset.take(round(DS_LEN*split)).batch(32)
valid = dataset.skip(round(DS_LEN*split)).batch(32)

In [11]:
len(train)

3902

In [12]:
from transformers import TFAutoModel, TFBertForSequenceClassification, BertConfig

In [13]:
# config = BertConfig.from_pretrained("bert-base-uncased", num_labels=5)
# bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", config=config)

In [14]:
# bert_model = TFAutoModel.from_pretrained('bert-base-uncased')
bert_model = TFAutoModel.from_pretrained('BAAI/bge-base-en-v1.5')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [15]:
bert_model.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
Total params: 109482240 (417.64 MB)
Trainable params: 109482240 (417.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
SQL_LEN = int(dataset.take(1).element_spec[0]['input_ids'].shape[0])

In [17]:
output_shape = int(dataset.take(1).element_spec[1].shape[0])

In [18]:
input_ids = tf.keras.layers.Input(shape=(SQL_LEN, ), name='input_ids', dtype=tf.int32)
mask = tf.keras.layers.Input(shape=(SQL_LEN, ), name='attention_mask', dtype=tf.int32)
bert_model.trainable = False
embeddings = bert_model.bert(input_ids=input_ids, attention_mask=mask)['pooler_output']
# x = tf.keras.layers.GlobalMaxPooling1D()(embeddings)
x = tf.keras.layers.Dense(50, activation='relu')(embeddings)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.Reshape((-1, x.shape[-1]))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False))(x)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
y = tf.keras.layers.Dense(output_shape, activation='softmax', name='outputs')(x)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 50)]                 0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 50)]                 0         []                            
 )                                                                                                
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
                             ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, 50, 76                                       

In [20]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-05, epsilon=1e-08, decay=0.01, clipnorm=1.0)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy()

In [21]:
model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [22]:
from tensorflow.keras.callbacks import TensorBoard

log_dir = './log'
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [23]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./model/v3',
    save_weights_only=True,
    monitor='val_categorical_accuracy',
    mode='max',
    save_best_only=True)

In [24]:
model.fit(train, validation_data=valid, epochs=10, verbose=1, callbacks=[
    tensorboard_callback, 
    cp_callback,
    tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-3 * 10 ** (epoch/3))
    ])

Epoch 1/10


2024-02-11 22:26:50.173531: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


3902/3902 [==============================] - 1916s 489ms/step - loss: 0.7879 - categorical_accuracy: 0.6558 - val_loss: 0.6721 - val_categorical_accuracy: 0.7154 - lr: 0.0010
Epoch 2/10
3902/3902 [==============================] - 1873s 480ms/step - loss: 0.7299 - categorical_accuracy: 0.6852 - val_loss: 0.6680 - val_categorical_accuracy: 0.7168 - lr: 0.0022
Epoch 3/10
3902/3902 [==============================] - 1868s 479ms/step - loss: 0.7310 - categorical_accuracy: 0.6864 - val_loss: 0.6556 - val_categorical_accuracy: 0.7227 - lr: 0.0046
Epoch 4/10
3902/3902 [==============================] - 1866s 478ms/step - loss: 0.7622 - categorical_accuracy: 0.6716 - val_loss: 0.6575 - val_categorical_accuracy: 0.7208 - lr: 0.0100
Epoch 5/10
3902/3902 [==============================] - 1885s 483ms/step - loss: 0.7453 - categorical_accuracy: 0.6765 - val_loss: 0.6786 - val_categorical_accuracy: 0.7093 - lr: 0.0215
Epoch 6/10
3902/3902 [==============================] - 1882s 482ms/step - loss: 

In [25]:
model.save('./model/v3.keras')